In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/million-trees/RELEASE_v0.2.txt
/kaggle/input/million-trees/crossgeometry.csv
/kaggle/input/million-trees/zeroshot.csv
/kaggle/input/million-trees/official.csv
/kaggle/input/million-trees/images/2018_SJER_3_258000_4106000_image_514.png
/kaggle/input/million-trees/images/05119e37-naips7-3584_2048_tl_6.png
/kaggle/input/million-trees/images/Pfinztal_6.png
/kaggle/input/million-trees/images/2018_SJER_3_258000_4106000_image_247.png
/kaggle/input/million-trees/images/6f08a338-naips7-6656_12800_tr_2.png
/kaggle/input/million-trees/images/2018_SJER_3_258000_4106000_image_231.png
/kaggle/input/million-trees/images/B07_0154.JPG
/kaggle/input/million-trees/images/2018_SJER_3_258000_4106000_image_51.png
/kaggle/input/million-trees/images/2018_SJER_3_259000_4110000_image_231.png
/kaggle/input/million-trees/images/B01_0135.JPG
/kaggle/input/million-trees/images/2018_SJER_3_258000_4106000_image_61.png
/kaggle/input/million-trees/images/2018_SJER_3_258000_4106000_image_343.png
/kaggle/in

In [2]:
!pip install MillionTrees

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.2/342.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00
   

# Creating a custom Loader


Since the MillionTrees datasest loader is not working


In [3]:
import os
import pandas as pd
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MillionTreesPoints(Dataset):
    def __init__(self,
                 annotations_csv: str,
                 images_dir: str,
                 split: str = None,
                 transform=None):
        # 1) Load CSV and pick the correct filename column
        self.df = pd.read_csv(annotations_csv, low_memory=False)
        if   "filename"   in self.df: self.file_col = "filename"
        elif "image_name" in self.df: self.file_col = "image_name"
        else: raise KeyError("No 'filename' or 'image_name' column found")

        # 2) Filter by split if requested
        if split and "split" in self.df:
            self.df = self.df[self.df["split"] == split]

        # 3) Convert labels to integer codes if present
        if "label" in self.df.columns:
            # ensure every entry is a string, then map to codes 0..C-1
            self.df["label"] = self.df["label"].astype(str)
            codes, uniques = pd.factorize(self.df["label"])
            self.df["label_idx"] = codes
            # keep a mapping if you want to decode later
            self.label_map = {idx: lbl for idx, lbl in enumerate(uniques)}
        else:
            self.label_map = None

        # 4) Group annotations by image file
        self.grouped = self.df.groupby(self.file_col)
        self.image_names = list(self.grouped.groups.keys())

        self.images_dir = images_dir
        self.transform  = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.images_dir, img_name)
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        annots = self.grouped.get_group(img_name)
        points = torch.from_numpy(annots[["x", "y"]].to_numpy()).float()

        if self.label_map is not None:
            labels = torch.from_numpy(annots["label_idx"].to_numpy()).long()
        else:
            labels = None

        return {
            "image":  img,
            "points": points,
            "labels": labels,
            "meta":   {self.file_col: img_name}
        }


In [4]:
csv_file  = "/kaggle/input/million-trees/official.csv"
images_dir = "/kaggle/input/million-trees/images"

In [5]:
# IMAGE TRANSFORMATIONS
tfm = transforms.Compose([
    transforms.Resize((448, 448)),
    transforms.ToTensor(),
])

ds = MillionTreesPoints(
    annotations_csv=csv_file,
    images_dir=images_dir,
    split="train",
    transform=tfm
)

# wrap in a DataLoader
loader = DataLoader(
    ds,
    batch_size=8,
    shuffle=True,
    collate_fn=lambda batch: batch
)

In [6]:
batch = next(iter(loader))
print("batch size:", len(batch))
print("keys:", batch[0].keys())
print("image shape:", batch[0]["image"].shape)
print("points shape:", batch[0]["points"].shape)
print("labels:", batch[0]["labels"])       # now a LongTensor or None
print("label_map:", ds.label_map)          # to decode ints→original strings


batch size: 8
keys: dict_keys(['image', 'points', 'labels', 'meta'])
image shape: torch.Size([3, 448, 448])
points shape: torch.Size([68, 2])
labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label_map: {0: 'Tree'}


In [7]:
import random
from pathlib import Path

# 1) Grab all image names from your dataset
all_images = ds.image_names 

# 2) Decide on a seed size
SEED_SIZE = 100  

# 3) Randomly sample your seed set
random.seed(42)  # for reproducibility
labeled = random.sample(all_images, SEED_SIZE)

# 4) The rest go into the unlabeled pool
unlabeled = [img for img in all_images if img not in labeled]

# 5) Persist these splits so you can inspect and resume later
Path("labeled.txt").write_text("\n".join(labeled))
Path("unlabeled.txt").write_text("\n".join(unlabeled))

# 6) Quick sanity printouts
print(f"Total images       : {len(all_images)}")
print(f"Initial labeled set : {len(labeled)}")
print(f"Initial unlabeled   : {len(unlabeled)}")


Total images       : 3529
Initial labeled set : 100
Initial unlabeled   : 3429


In [8]:
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image

class ActiveMillionTreesDataset(Dataset):
    def __init__(self, base_ds, image_list, transform=None):
        """
        base_ds    : your MillionTreesPoints instance
        image_list : list of filenames (e.g., from labeled.txt or unlabeled.txt)
        transform  : torchvision transform to apply
        """
        df = base_ds.df[base_ds.df[base_ds.file_col].isin(image_list)]
        self.grouped     = df.groupby(base_ds.file_col)
        self.image_names = list(self.grouped.groups.keys())
        self.images_dir  = base_ds.images_dir
        self.transform   = transform
        self.file_col    = base_ds.file_col

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = Path(self.images_dir) / img_name
        img      = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        annots = self.grouped.get_group(img_name)
        points = torch.from_numpy(annots[["x","y"]].to_numpy()).float()
        labels = (torch.from_numpy(annots["label_idx"].to_numpy()).long()
                  if "label_idx" in annots.columns else None)

        return {
            "image":  img,
            "points": points,
            "labels": labels,
            "meta":   {self.file_col: img_name}
        }


In [9]:

# read back your splits (if you saved them to disk)
labeled   = Path("labeled.txt").read_text().splitlines()
unlabeled = Path("unlabeled.txt").read_text().splitlines()

# define your transforms (the same you used before)
tfm = transforms.Compose([
    transforms.Resize((448,448)),
    transforms.ToTensor(),
])

# seed (labeled) dataset & loader
seed_ds    = ActiveMillionTreesDataset(ds, labeled,   transform=tfm)
seed_loader = DataLoader(seed_ds, batch_size=8, shuffle=True,  collate_fn=lambda x: x)

# pool (unlabeled) dataset & loader
pool_ds    = ActiveMillionTreesDataset(ds, unlabeled, transform=tfm)
pool_loader = DataLoader(pool_ds, batch_size=1, shuffle=False, collate_fn=lambda x: x)

# sanity‑check sizes
print("Seed images    :", len(seed_ds))
print("Unlabeled pool :", len(pool_ds))


Seed images    : 100
Unlabeled pool : 3429
